# Lesson 4: Deploy an AWS Lambda function

### Import all needed packages

In [1]:
import boto3, os

In [2]:
from helpers.Lambda_Helper import Lambda_Helper
from helpers.S3_Helper import S3_Helper
from helpers.Display_Helper import Display_Helper

In [3]:
lambda_helper = Lambda_Helper()
# deploy_function
# add_lambda_trigger

In [4]:
s3_helper = S3_Helper()
# upload_file
# download_object 
# list_objects

In [5]:
display_helper = Display_Helper()
# text_file
# json_file

In [6]:
bucket_name_text = os.environ['LEARNERS3BUCKETNAMETEXT']

In [7]:
%%writefile prompt_template.txt

I need to summarize a conversation. The transcript of the conversation is between the <data> XML like tags.

<data>
{{transcript}}
</data>

The summary must contain a one word sentiment analysis, and a list of issues, problems or causes of friction
during the conversation. The output must be provided in JSON format shown in the following example. 

Example output:
{
    "version": 0.1,
    "sentiment": <sentiment>,
    "issues": [
        {
            "topic": <topic>,
            "summary": <issue_summary>,
        }
    ]
}

An `issue_summary` must only be one of:
{%- for topic in topics %}
 - `{{topic}}`
{% endfor %}

Write the JSON output and nothing more.

Here is the JSON output:

Writing prompt_template.txt


In [8]:
display_helper.text_file('prompt_template.txt')

prompt_template.txt:


### Create the Lambda function

In [9]:
%%writefile lambda_function.py


#############################################################
#
# This Lambda function is written to a file by the notebook 
# It does not run in the notebook!
#
#############################################################

import boto3
import json 
from jinja2 import Template

s3_client = boto3.client('s3')
bedrock_runtime = boto3.client('bedrock-runtime', 'us-west-2')

def lambda_handler(event, context):
    
    bucket = event['Records'][0]['s3']['bucket']['name']
    key = event['Records'][0]['s3']['object']['key']
    
    # One of a few different checks to ensure we don't end up in a recursive loop.
    if "-transcript.json" not in key: 
        print("This demo only works with *-transcript.json.")
        return
    
    try: 
        file_content = ""
        
        response = s3_client.get_object(Bucket=bucket, Key=key)
        
        file_content = response['Body'].read().decode('utf-8')
        
        transcript = extract_transcript_from_textract(file_content)

        print(f"Successfully read file {key} from bucket {bucket}.")

        print(f"Transcript: {transcript}")
        
        summary = bedrock_summarisation(transcript)
        
        s3_client.put_object(
            Bucket=bucket,
            Key='results.txt',
            Body=summary,
            ContentType='text/plain'
        )
        
    except Exception as e:
        print(f"Error occurred: {e}")
        return {
            'statusCode': 500,
            'body': json.dumps(f"Error occurred: {e}")
        }

    return {
        'statusCode': 200,
        'body': json.dumps(f"Successfully summarized {key} from bucket {bucket}. Summary: {summary}")
    }
        
        
        
def extract_transcript_from_textract(file_content):

    transcript_json = json.loads(file_content)

    output_text = ""
    current_speaker = None

    items = transcript_json['results']['items']

    # Iterate through the content word by word:
    for item in items:
        speaker_label = item.get('speaker_label', None)
        content = item['alternatives'][0]['content']
        
        # Start the line with the speaker label:
        if speaker_label is not None and speaker_label != current_speaker:
            current_speaker = speaker_label
            output_text += f"\n{current_speaker}: "
        
        # Add the speech content:
        if item['type'] == 'punctuation':
            output_text = output_text.rstrip()  # Remove the last space
        
        output_text += f"{content} "
        
    return output_text
        

def bedrock_summarisation(transcript):
    
    with open('prompt_template.txt', "r") as file:
        template_string = file.read()

    data = {
        'transcript': transcript,
        'topics': ['charges', 'location', 'availability']
    }
    
    template = Template(template_string)
    prompt = template.render(data)
    
    print(prompt)
    
    kwargs = {
        "modelId": "amazon.titan-text-express-v1",
        "contentType": "application/json",
        "accept": "*/*",
        "body": json.dumps(
            {
                "inputText": prompt,
                "textGenerationConfig": {
                    "maxTokenCount": 2048,
                    "stopSequences": [],
                    "temperature": 0,
                    "topP": 0.9
                }
            }
        )
    }
    
    response = bedrock_runtime.invoke_model(**kwargs)

    summary = json.loads(response.get('body').read()).get('results')[0].get('outputText')    
    return summary
    
    

Writing lambda_function.py


In [10]:
lambda_helper.deploy_function(
    ["lambda_function.py", "prompt_template.txt"],
    function_name="LambdaFunctionSummarize"
)

Zipping function...
Looking for existing function...
Function LambdaFunctionSummarize does not exist. Creating...
Function LambdaFunctionSummarize created: arn:aws:lambda:us-west-2:433586054185:function:LambdaFunctionSummarize
Done.


In [11]:
lambda_helper.filter_rules_suffix = "json"
lambda_helper.add_lambda_trigger(bucket_name_text)

Using function name of deployed function: LambdaFunctionSummarize
Permission added with Statement: {
    "Sid": "s3-trigger-permission",
    "Effect": "Allow",
    "Principal": {
        "Service": "s3.amazonaws.com"
    },
    "Action": "lambda:InvokeFunction",
    "Resource": "arn:aws:lambda:us-west-2:433586054185:function:LambdaFunctionSummarize",
    "Condition": {
        "ArnLike": {
            "AWS:SourceArn": "arn:aws:s3:::c99355a2566046l6701965t1w781388481-learners3bucket-ycosghfxbvq3"
        }
    }
}
Trigger added for c99355a2566046l6701965t1w781388481-learners3bucket-ycosghfxbvq3 -> LambdaFunctionSummarize


In [ ]:
# display_helper.json_file('demo-transcript.json')

In [12]:
s3_helper.upload_file(bucket_name_text, 'demo-transcript.json')

Object 'demo-transcript.json' uploaded to bucket 'c99355a2566046l6701965t1w781388481-learners3bucket-ycosghfxbvq3'


#### Restart kernel if needed.
- If you run the code fairly quickly from start to finish, it's possible that the following code cell `s3_helper.list_objects(bucket_name_text)` will give a "Not Found" error.  
- If waiting a few seconds (10 seconds) and re-running this cell does not resolve the error, then you can restart the kernel of the jupyter notebook.
- Go to menu->Kernel->Restart Kernel.
- Then run the code cells from the start of the notebook, waiting 2 seconds or so for each code cell to finish executing.

In [13]:
s3_helper.list_objects(bucket_name_text)

Object: demo-transcript.json, Created on: 2025-08-24 07:49:12+00:00
Object: results.txt, Created on: 2025-08-24 07:49:19+00:00


#### Re-run "download" code cell as needed
- It may take a few seconds for the results to be generated.
- If you see a `Not Found` error, please wait a few seconds and then try running the `s3_helper.download_object` again.

In [14]:
s3_helper.download_object(bucket_name_text, "results.txt")

Object 'results.txt' from bucket 'c99355a2566046l6701965t1w781388481-learners3bucket-ycosghfxbvq3' to './results.txt'


In [15]:
display_helper.text_file('results.txt')

results.txt:
